In [1]:
import numpy as np
import pandas as pd
import math
import copy
import re
import nltk
import sklearn
import os
from scipy import spatial
import csv

In [2]:
correlationThreshold = 0.35

**To handle file names from 2000**

In [3]:
def expand(file):
    file = str(file)
    if len(file) == 7:
        return str(file)
    else:
        num = 7-len(file)
        for i in range(0,num):
            file = str(0) + file
        return str(file)

**Check if BERT embeddings is present..if not skip the entire **

In [4]:
def BertChecker(path):
    newPath = []
    bertFolder = r'C:\Users\rishe\Desktop\DMproject2\Bert\\'
    for files in path:
        fname = bertFolder + expand(files) + '.csv'
        if os.path.isfile(fname) == True:
          #print(files)
            newPath.append(files) 

    return newPath

**Generate top 5 topics**

In [5]:
pathBert = r'C:\Users\rishe\Desktop\DMproject2\Bert\\'
pathExtractedTopics = r'C:\Users\rishe\Desktop\DMproject2\ExtractedTopics\\'

In [6]:
def writeToCSV(matrix,file1,file2):
    path = r'C:\Users\rishe\Desktop\DMproject2\Correlation-deepscore\\'
    df=pd.DataFrame(matrix)
    path = path + str(file1) + '-' + str(file2) + '.csv'
    df.to_csv(path, encoding='utf-8')

In [7]:
def cond(x):
    return x[2]

In [8]:
def topFive(matrix):
    qSize = 5
    queue = []
    for i in range(qSize):
        arr = []
        arr.append(0)
        arr.append(i)
        arr.append(matrix[0][i])
        queue.append(arr)
    queue.sort(key = cond)
  
    for i in range(qSize,len(matrix[0])):
        if matrix[0][i] < cond(queue[0]):
            continue
        else:
            queue.pop(0)
            arr = []
            arr.append(0)
            arr.append(i)
            arr.append(matrix[0][i])
            queue.append(arr)
            queue.sort(key = cond)

    for j in range(1,len(matrix)):
        for k in range(j,len(matrix[j])):
            if matrix[j][k] < cond(queue[0]):
                continue
            else:
                queue.pop(0)
                arr = []
                arr.append(j)
                arr.append(k)
                arr.append(matrix[j][k])
                queue.append(arr)
                queue.sort(key = cond)

  #print(queue)
    return queue

In [9]:
def writeTopFive(file1,file2,score,q):
    lout = []
    lout.append(file1)
    lout.append(file2)
    lout.append(score)
    lout.append(str(q))

    fd = r'C:\Users\rishe\Desktop\DMproject2\Output\topFive-deepscore.csv'
    with open(fd,'a') as f:
        write = csv.writer(f)
        write.writerow(lout)

In [10]:
def formNetwork(file1,file2,score):

    file1 = expand(file1)
    file2 = expand(file2)
    Pbert1 = pathBert + str(file1) + '.csv' 
    Pbert2 = pathBert + str(file2) + '.csv'
    Ptopics1 = pathExtractedTopics  + str(file1) + '.csv'
    Ptopics2 = pathExtractedTopics  + str(file2) + '.csv'

    try:
        bertEmbed1 = pd.read_csv(Pbert1).values
        bertEmbed2 = pd.read_csv(Pbert2).values
        # topics1 = pd.read_csv(Ptopics1).values
        # topics2 = pd.read_csv(Ptopics2).values
    
    except:
        lout = []
        lout.append(file1)
        lout.append(file2)

        #print(lout)
        #cleaned=pd.DataFrame(lout)
        fd = r'C:\Users\rishe\Desktop\DMproject2\Error\Error-BertNotFound.csv'
        with open(fd,'a') as f:
            write = csv.writer(f)
            write.writerow(lout)
        print("error-Topic or BERT file not found" + file1 + " " + file2)
        return
  
    bertEmbed1 = bertEmbed1[:,1:]
    bertEmbed2 = bertEmbed2[:,1:]
  
    n1=(len(bertEmbed1))
    n2=(len(bertEmbed2))
    corr=[[-999 for i in range(n1)]for j in range(n2)]
    for i in range(n1):
        for j in range(n2):
            corr[i][j]=round(spatial.distance.correlation(bertEmbed1[i],bertEmbed2[j]),3)
  
    q = topFive(corr)
    writeToCSV(corr,file1,file2)
    writeTopFive(file1,file2,score,q)
    #return q

**Convert The top 5 topics to a 2-D array**

In [11]:
def topicsArray(s, reverse):
    t = s[1:-1]
    t = t.split("], ")
    k = []
    for i in t:
        i = i.replace("[","")
        i = i.replace(']',"")
        i = i.split(", ")
        l = []

        if reverse == 0:
            for j in range(0,2):
                l.append(int(i[j]))

        elif reverse == 1:
            l.append(i[1])
            l.append(i[0])
 
        k.append(l)
    
    return k

To form parent array on which further appending will take place, to run only on the first pair of each array in LISD 

In [12]:
def topicAdder(superArray, array, start, destination):
    arr = superArray
    newArr = []
    for i in arr:
        number = i[-1]
        if number == -1:
            a = copy.deepcopy(i)
            a.append(-1)
            newArr.append(a)
            continue
        count = 0
        for j in array:
            if j[0] == number:
                a=[]
                count = count + 1
                a = copy.deepcopy(i) 
                a.append(j[1])
                newArr.append(a)
        if count == 0:
            try:
                path = r'C:\Users\rishe\Desktop\DMproject2\Correlation-deepscore\\' + expand(start) + '-' + expand(destination) + '.csv'
                value = pd.read_csv(path).values
                row = number
                #print(value[row] , number)
                max = -99999
                index = -1
                for column in range(1,len(value[row])):
                    if value[row][column] > max and value[row][column] >= correlationThreshold:
                        index = column-1
                        max = value[row][column]
                a = copy.deepcopy(i)
                a.append(index)
                newArr.append(a)
            except:
                path = r'C:\Users\rishe\Desktop\DMproject2\Correlation-deepscore\\' + expand(destination) + '-' + expand(start) + '.csv'
                value = pd.read_csv(path).values
                row = number
                #print(value[row] , number)
                max = -99999
                index = -1
                for column in range(1,len(value[row])):
                    if value[row][column] > max and value[row][column] >= correlationThreshold:
                        index = column-1
                        max = value[row][column]
                a = copy.deepcopy(i)
                a.append(index)
                newArr.append(a)

    return newArr

**To form SuperArray**

In [13]:
def relationCheckFirst(start,destination):
    topTopics = pd.read_csv(r'C:\Users\rishe\Desktop\DMproject2\Output\topFive-deepscore.csv').values
    for i in range(len(topTopics)):
        if topTopics[i][0] == int(start) and topTopics[i][1] == int(destination):
            s = topTopics[i][3] 
            array = topicsArray(s,0)
            return array

        elif topTopics[i][0] == int(destination) and topTopics[i][1] == int(start):
            s = topTopics[i][3] 
            array = topicsArray(s,1)
            return array

    formNetwork(int(start),int(destination),1.11)
    return relationCheckFirst(start,destination)

**To append to SuperArray**

In [14]:
def relationCheckLast(start,destination,superArray):
    topTopics = pd.read_csv(r'C:\Users\rishe\Desktop\DMproject2\Output\topFive-deepscore.csv').values
    for i in range(len(topTopics)):
        if topTopics[i][0] == int(start) and topTopics[i][1] == int(destination):
            s = topTopics[i][3] 
            array = topicsArray(s,0)
            return topicAdder(superArray, array,start=start, destination=destination)

        elif topTopics[i][0] == int(destination) and topTopics[i][1] == int(start):
            s = topTopics[i][3] 
            array = topicsArray(s,1)
            return topicAdder(superArray, array,start=start, destination=destination)

    formNetwork(int(start),int(destination),1.11)
    return relationCheckLast(start,destination,superArray)

In [15]:
def writeNetwork(path,topics):
    lout = []
    lout.append(str(path))
    lout.append(str(topics))

    fd = r'C:\Users\rishe\Desktop\DMproject2\Output\Path-Topics.csv'
    with open(fd,'a') as f:
        write = csv.writer(f)
        write.writerow(lout)

**RUN**

In [16]:
relations = pd.read_csv(r'C:\Users\rishe\Desktop\DMproject2\Lineage2.csv').values
topTopics = pd.read_csv(r'C:\Users\rishe\Desktop\DMproject2\Output\topFive-deepscore.csv').values

In [17]:
lisd=[]
for i in range(len(relations)):
    try:
        temp= relations[i][1].split("[")
        temp=temp[1].split("]")
        temp=temp[0].split(" ") 
        lisd.append(temp)
        # print(temp)
    except:
        continue


In [18]:
print(len(lisd))

757


In [19]:
lisd

[['9207071', '9601070', '9904105', '5187', '105305', '208087'],
 ['9410188',
  '9503157',
  '9709138',
  '9711001',
  '9805191',
  '9807040',
  '9912185',
  '12145',
  '106048',
  '209131',
  '211209'],
 ['9201036', '9307176', '9508030', '9610224', '9805160', '212084'],
 ['9408062',
  '9512066',
  '9612114',
  '203020',
  '204054',
  '208074',
  '209179',
  '301009'],
 ['9311081',
  '9505052',
  '9506106',
  '9512031',
  '9607026',
  '9609017',
  '9612157',
  '9703030',
  '9707093',
  '9803196',
  '9807113',
  '9812095',
  '9907152',
  '211257'],
 ['9304045',
  '9601108',
  '9604034',
  '9706109',
  '9707093',
  '9711200',
  '9712251',
  '9806026',
  '9806194',
  '9902098',
  '8170',
  '8184'],
 ['9202059',
  '9703206',
  '9707126',
  '9710151',
  '9811047',
  '9908160',
  '11086',
  '106015',
  '205066',
  '206059',
  '301250'],
 ['9211121',
  '9505123',
  '9607058',
  '9701015',
  '9706047',
  '9801134',
  '9808075',
  '9905111',
  '9906056',
  '9906105',
  '5081',
  '6239',
  '7067'

In [21]:
count = 0
for i in range(len(lisd)):
    try: 
        path = BertChecker(lisd[i])
        #print("Bert files found")
        superArray = relationCheckFirst(path[0] , path[1])
        begin=1
        end=begin+1
        while end<len(path):
            start=expand(path[begin])
            dest=expand(path[end])
            superArray = relationCheckLast(start,dest,superArray)
            begin = begin+1
            end = end+1
        writeNetwork(path,superArray)

    except:
        fd = r'C:\Users\rishe\Desktop\DMproject2\Error\Error-PathFinder-2.csv'
        with open(fd,'a') as f:
            write = csv.writer(f)
            write.writerow(lisd[i])

    count = count+1
    if count%5 == 0:
        print(count)

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755


In [22]:
i = 0
try: 
    print(lisd[i])
    path = BertChecker(lisd[i])
    print(path)
    print(path[0] + " " + path[1])
    superArray = relationCheckFirst(path[0] , path[1])
    print(superArray)
    begin=1
    end=begin+1
    while end<len(path):
        start=expand(path[begin])
        dest=expand(path[end])
        print(start + " " + dest)
        superArray = relationCheckLast(start,dest,superArray)
        print(superArray)
        begin = begin+1
        end = end+1
    writeNetwork(path,superArray)

except:
    fd = r'C:\Users\rishe\Desktop\DMproject2\Error\Error-PathFinder.csv'
    with open(fd,'a') as f:
        write = csv.writer(f)
        write.writerow(lisd[i])
    print("error - Not able find files for path")

['9207071', '9601070', '9904105', '5187', '105305', '208087']
['9207071', '9601070', '9904105', '5187', '105305', '208087']
9207071 9601070
[[2, 15], [5, 15], [3, 13], [3, 15], [3, 11]]
9601070 9904105
[[2, 15, 18], [2, 15, 15], [2, 15, 19], [5, 15, 18], [5, 15, 15], [5, 15, 19], [3, 13, 2], [3, 15, 18], [3, 15, 15], [3, 15, 19], [3, 11, 2]]
9904105 0005187
[[2, 15, 18, 6], [2, 15, 15, 19], [2, 15, 19, 16], [5, 15, 18, 6], [5, 15, 15, 19], [5, 15, 19, 16], [3, 13, 2, 3], [3, 15, 18, 6], [3, 15, 15, 19], [3, 15, 19, 16], [3, 11, 2, 3]]
0005187 0105305
[[2, 15, 18, 6, 14], [2, 15, 15, 19, 14], [2, 15, 19, 16, 16], [5, 15, 18, 6, 14], [5, 15, 15, 19, 14], [5, 15, 19, 16, 16], [3, 13, 2, 3, 19], [3, 15, 18, 6, 14], [3, 15, 15, 19, 14], [3, 15, 19, 16, 16], [3, 11, 2, 3, 19]]
0105305 0208087
[[2, 15, 18, 6, 14, 9], [2, 15, 15, 19, 14, 9], [2, 15, 19, 16, 16, 6], [5, 15, 18, 6, 14, 9], [5, 15, 15, 19, 14, 9], [5, 15, 19, 16, 16, 6], [3, 13, 2, 3, 19, 9], [3, 15, 18, 6, 14, 9], [3, 15, 15, 19

In [ ]:
i=0
path = BertChecker(lisd[i])
print(path)

9207071
9601070
9904105
5187
105305
208087
['9207071', '9601070', '9904105', '5187', '105305', '208087']


In [ ]:
b = relationCheckFirst(9207071,9601070)
b

[[2, 15], [5, 15], [3, 13], [3, 15], [3, 11]]

In [ ]:
c = topicAdder(a,b)
c

[[11, 15, 16], [4, 13, -1], [4, 19, -1], [11, 13, -1], [4, 15, 16]]

In [ ]:
d = relationCheckFirst(9508030,9610224)
d

[[12, 12], [2, 12], [1, 12], [9, 12], [11, 12]]

In [ ]:
e = topicAdder(c,d)
e

[[11, 15, 16, -1],
 [4, 13, -1, -1],
 [4, 19, -1, -1],
 [11, 13, -1, -1],
 [4, 15, 16, -1]]

In [ ]:
BertChecker(lisd[0])

True